In [ ]:
# using Pkg; Pkg.update("TuLiPa") # uncomment to update TuLiPa to latest version
# using DataFrames, Plots, Statistics, JSON, TuLiPa, Dates, HiGHS, JuMP
using DataFrames, Plots, Statistics, JSON, Dates, HiGHS, JuMP, TuLiPa
# include(joinpath(dirname(dirname(dirname(pwd()))),"jgrc/TuLiPa/src/TuLiPa.jl"));
include(joinpath(dirname(dirname(pwd())),"src/util.jl"));
plotlyjs() # uncomment for interactive plots

In [ ]:
using JLD2

In [ ]:
prob = load("probGtfW4MoTGXM9.jld2")
p = prob["p"];

In [ ]:
env = TuLiPa._CPLEXEnv()
lp = TuLiPa._cplex_create_lp(env)
CPLEX.CPXreadcopyprob(env, lp, "failed_modelGtfW4MoTGXM9.mps", "MPS")
CPLEX.CPXsetintparam(env, 1062, 2) # 1/primal, 2/dual, 3/network, 4/barrier
# CPLEX.CPXsetintparam(env, 1147, 2) # solution type with barrier
# CPLEX.CPXsetintparam(env, 3017, 4) # barrier start alg
CPLEX.CPXsetintparam(env, 1035, 1) # Screenoutput
CPLEX.CPXlpopt(env, lp)  
objval_p = Ref{Cdouble}()
CPLEX.CPXgetobjval(env, lp, objval_p)
print(objval_p[])
display(CPLEX.CPXgetstat(env, lp))

In [ ]:
h = Highs_create()
Highs_readModel(h, "failed_modelGtfW4MoTGXM9.mps")
Highs_setIntOptionValue(h, "simplex_scale_strategy", 5)
ret = Highs_run(h)
Highs_getScaledModelStatus(p.inner)

In [ ]:
p.inner = Highs_create()
TuLiPa._passLP!(p)
Highs_setIntOptionValue(p, "simplex_scale_strategy", 5)
# Highs_setDoubleOptionValue(p, "primal_feasibility_tolerance", 1e-6) 
Highs_run(p.inner)
Highs_getScaledModelStatus(p.inner)

In [ ]:
# The hydropower storages in the dataset needs boundary conditions for the state variables
function addStartEqualStopAllStorages!(modelobjects)
    for obj in values(modelobjects)
        if obj isa BaseStorage
            trait = StartEqualStop(obj)
            modelobjects[getid(trait)] = trait
        end
    end
end

# Power balances needs slack variable for when the inelastic supply (wind, solar, RoR) is higher than the inelastic demand
function addPowerUpperSlack!(modelobjects) # add after object manipulation
    for obj in values(modelobjects)
        if obj isa BaseBalance
            if getid(getcommodity(obj)) == Id("Commodity", "Power")
                balancename = getinstancename(getid(obj))
                
                varname = "SlackVar_" * balancename
                varkey = Id(FLOW_CONCEPT, varname)
                var = BaseFlow(varkey)
                
                sethorizon!(var, gethorizon(obj))
                setlb!(var, LowerZeroCapacity())
                
                arrowname = "SlackArrow_" * balancename
                arrowkey = Id(ARROW_CONCEPT, arrowname) 
                arrow = BaseArrow(arrowkey, obj, BaseConversion(PlusOneParam()), 0)
                addarrow!(var, arrow)
                
                modelobjects[varkey] = var
            end
        end 
    end
end

# Remove elements that are not compatible with certain Horizons.
function remove_startupcosts!(modelobjects)
    for (id,obj) in modelobjects
        if obj isa StartUpCost
            delete!(modelobjects, id)
        end
    end
end
function remove_transmissionramping!(modelobjects::Dict)
    for (id,obj) in modelobjects
        if obj isa TransmissionRamping
            delete!(modelobjects, id)
        end
    end
end;

In [ ]:
weekstart = 46
config = JSON.parsefile(joinpath(dirname(pwd()), "jules_config.json"), use_mmap=false)
prognoser_path = joinpath(config[1], "prognosermodell", "prognoser")

sti_data = joinpath(prognoser_path, "static_input")
sti_data1 = joinpath(prognoser_path, "Uke_$(weekstart)", "input");

In [ ]:
exd = JSON.parsefile(joinpath(sti_data1, "exogenprices_prognose1.json"))
exogen = getelements(exd, sti_data1);

In [ ]:
add = JSON.parsefile(joinpath(sti_data, "aggdetd2.json"))
aggdetd = getelements(add, sti_data);

In [ ]:
ipad = JSON.parsefile(joinpath(sti_data1, "tilsigsprognoseragg1993.json"))
agginflow = getelements(ipad, sti_data1);

In [ ]:
dse = JSON.parsefile(joinpath(sti_data, "tidsserier_detd.json"))
detdseries = getelements(dse, sti_data);

In [ ]:
dda = JSON.parsefile(joinpath(sti_data, "dataset_detd.json"))
detdstructure = getelements(dda);

In [ ]:
ipd = JSON.parsefile(joinpath(sti_data1, "tilsigsprognoser1993.json"))
inflow = getelements(ipd, sti_data1);

In [ ]:
thd = JSON.parsefile(joinpath(sti_data, "termisk1.json"))
thermal = getelements(thd, sti_data);

In [ ]:
wsd = JSON.parsefile(joinpath(sti_data, "vindsol.json"))
windsol = getelements(wsd, sti_data);

In [ ]:
trd = JSON.parsefile(joinpath(sti_data1, "nett.json"))
transm = getelements(trd);

In [ ]:
cod = JSON.parsefile(joinpath(sti_data, "forbruk5.json"))
cons = getelements(cod, sti_data);

In [ ]:
fpd = JSON.parsefile(joinpath(sti_data1, "brenselspriser.json"))
fuel = getelements(fpd, sti_data1);

In [ ]:
nud = JSON.parsefile(joinpath(sti_data1, "nuclear.json"))
nuclear = getelements(nud, sti_data1);

In [ ]:
# tnd = JSON.parsefile(joinpath(sti_data, "termisk_nonucl.json"))
# thermal_nonucl = getelements(tnd, sti_data);

In [ ]:
# Gamle filer
# elements = vcat(exogen,detdseries,detdstructure,thermal,windsol,transm,cons,inflow)
# elements = vcat(exogen,aggdetd,thermal,windsol,transm,cons,agginflow)

# Filer med kjernekraft og fuel separat
elements = vcat(exogen,aggdetd,thermal,windsol,transm,cons,agginflow,nuclear,fuel)
# elements = vcat(exogen,detdseries,detdstructure,windsol,transm,cons,inflow,nuclear,fuel)

# Filer med uten kjernekraftprognose
# elements = vcat(exogen,aggdetd,thermal,windsol,transm,cons,agginflow,nuclear,fuel)
# elements = vcat(exogen,detdseries,thermal_nonucl,detdstructure,windsol,transm,cons,inflow,fuel)
length(elements)

In [ ]:
scenarioyearstart = 1991
scenarioyearstop = 2021
push!(elements, getelement(TIMEPERIOD_CONCEPT, "ScenarioTimePeriod", "ScenarioTimePeriod", 
        ("Start", getisoyearstart(scenarioyearstart)), ("Stop", getisoyearstart(scenarioyearstop))))

cnph = 26
cnpp = 26
cpdh = Hour(168*2)
cpdp = Hour(168*2)
# cnph = 24
# cnpp = 24
# cpdh = Hour(2)
# cpdp = Hour(2)
hydro_horizon = SequentialHorizon(cnph, cpdh)
power_horizon = SequentialHorizon(cnpp, cpdp)

# Insert horizons into commodities. E.g. all batteries will have the power horizon, since they interact with the power market
function set_horizon!(elements, commodity, horizon)
    # If element already exist, replace horizon with new
    for element in elements
        if element.typename == "BaseCommodity"
            if element.instancename == commodity
                element.value[HORIZON_CONCEPT] = horizon
                return
            end
        end
    end
    
    # Else, add commodity to element list
    push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", commodity, 
        (HORIZON_CONCEPT, horizon)))
end

# The power horizon will be assigned to everything contributing to the power market, or batteries
set_horizon!(elements, "Power", power_horizon)
set_horizon!(elements, "Battery", power_horizon)

# The hydro horizon will be assigned to storages, bypasses and spill variables of hydropower plants (not release because it contributes to the power market and therefore needs to have power_horizon as its horizon)
set_horizon!(elements, "Hydro", hydro_horizon);

In [ ]:
modelobjects = getmodelobjects(elements);
length(modelobjects)

### Simplify market description and add boundary conditions
When the model objects have been created we can manipulate them however we want. In this example we simplify the problem by aggregating areas and power plants, and removing short term storage systems and start-up costs. We also add boundary conditions to storages.

In [ ]:
function simplify!(modelobjects)    
    # Add slack variable for excessive renewable power
    addPowerUpperSlack!(modelobjects)

    # Start-up-costs are not compatible with aggregatesupplycurve! or AdaptiveHorizon
    # remove_startupcosts!(modelobjects)
    
    # TransmissionRamping not compatible with AdaptiveHorizon
    remove_transmissionramping!(modelobjects)

    # Aggregate all simple plants (only connected to power market, mostly thermal) for each area into 4 equivalent plants
    # aggregatesupplycurve!(modelobjects, 4)

    # Short-term storage systems are only needed when the horizon is fine 
    # removestoragesystems!(modelobjects, Hour(10))

    # # Only calculate AdaptiveHorizon based on residual loads in these areas
    # residualloadareas!(modelobjects, ["","NLDBEL","POL","GBR","BAL"])

    # Storages have state-dependant variables that need a boundary conditions
    # We set the starting storage to be equal to the ending storage, x[0] = x[T] (for horizon where t in 1:T)
    # addStartEqualStopAllStorages!(modelobjects)
end

simplify!(modelobjects);

### Solve a problem and plot results
Due to AdaptiveHorizon we get hourly price volatility even though the LP-problem only has seven periods per week. This leads to seven price levels per week, where hours with similar residual loads will have the same price, because they where grouped together into one period.

In [ ]:
# Choose scenarios
@time begin
    @time prob = buildprob(CPLEXIPMMethod(), modelobjects)
    # @time prob = buildprob(HighsSimplexMethod(), collect(values(modelobjects)))

    weekstart = 45
    t = PrognosisTime(getisoyearstart(2023) + Week(weekstart-1), getisoyearstart(2025) + Week(weekstart-1), getisoyearstart(1992) + Week(weekstart-1))

    storages = getstorages(getobjects(prob))
    setstartstoragepercentage!(prob, storages, t, 70.0) # replace with user settings
    setendstoragepercentage!(prob, storages, t, 70.0)

    @time update!(prob, t)

    @time solve!(prob)

    t += Day(1)

    @time update!(prob, t)

    @time solve!(prob)
end
println(getobjectivevalue(prob))

# Choose areas to see results from
probobjects = Dict(zip([getid(obj) for obj in prob.objects],prob.objects)) # collect results from all areas
# resultobjects = getpowerobjects(modelobjects,["SORLAND", "FINNMARK","SVER-SE3","DANM-DK1"]) #,"NOS","DMK","DEU", "FRACHE"
resultobjects = prob.objects # collect results for all areas

@time results = init_results(prob, probobjects, resultobjects, cnpp, cnph, cpdp, t, true);
prices, rhstermvalues, production, consumption, hydrolevels, batterylevels, powerbalances, rhsterms, rhstermbalances, plants, plantbalances, plantarrows, demands, demandbalances, demandarrows, hydrostorages, batterystorages = results

In [ ]:
# Only keep rhsterms that have at least one value (TODO: Do the same for sypply and demands)
rhstermtotals = dropdims(sum(rhstermvalues,dims=1),dims=1)
rhstermsupplyidx = []
rhstermdemandidx = []

for k in 1:length(rhsterms)
    if rhstermtotals[k] > 0
        push!(rhstermsupplyidx, k)
    elseif rhstermtotals[k] < 0
        push!(rhstermdemandidx, k)
    end
end

# Put rhsterms together with supplies and demands
rhstermsupplyvalues = rhstermvalues[:,rhstermsupplyidx]
rhstermdemandvalues = rhstermvalues[:,rhstermdemandidx]*-1

rhstermsupplynames = [getinstancename(rhsterm) for rhsterm in rhsterms[rhstermsupplyidx]]
rhstermsupplybalancenames = [split(getinstancename(r), "PowerBalance_")[2] for r in rhstermbalances[rhstermsupplyidx]]
rhstermdemandnames = [getinstancename(rhsterm) for rhsterm in rhsterms[rhstermdemandidx]]
rhstermdemandbalancenames = [split(getinstancename(r), "PowerBalance_")[2] for r in rhstermbalances[rhstermdemandidx]]

supplynames = [[getinstancename(plant) for plant in plants];rhstermsupplynames]
supplybalancenames = [[split(getinstancename(p), "PowerBalance_")[2] for p in plantbalances];rhstermsupplybalancenames]
supplyvalues = hcat(production,rhstermsupplyvalues)

demandnames = [[getinstancename(demand) for demand in demands];rhstermdemandnames]
demandbalancenames = [[split(getinstancename(p), "PowerBalance_")[2] for p in demandbalances];rhstermdemandbalancenames]
demandvalues = hcat(consumption, rhstermdemandvalues)

# Prepare for plotting results
hydronames = [getinstancename(hydro) for hydro in hydrostorages]
batterynames = [getinstancename(battery) for battery in batterystorages]
powerbalancenames = [split(getinstancename(getid(powerbalance)), "PowerBalance_")[2] for powerbalance in powerbalances]

# Convert reservoir filling to TWh
hydrolevels1 = copy(hydrolevels)
for (i,hydroname) in enumerate(hydronames)
    if haskey(getbalance(probobjects[hydrostorages[i]]).metadata, GLOBALENEQKEY)
        hydrolevels1[:,i] .= hydrolevels1[:,i]*getbalance(probobjects[hydrostorages[i]]).metadata[GLOBALENEQKEY]
    end
end

# Time
x1 = [getisoyearstart(scenarioyearstart) + cpdp*(t-1) for t in 1:first(size(supplyvalues))] # power/load resolution
x2 = [getisoyearstart(scenarioyearstart) + cpdh*(t-1) for t in 1:first(size(hydrolevels))]; # reservoir resolution

In [ ]:
# Plot prices
idxwohub = findall(x -> !occursin("HUB", x), powerbalancenames) # remove hubs, not active in 2025 dataset
display(plot(x1, prices[:,idxwohub]*100, labels=reshape(powerbalancenames[idxwohub],1,length(powerbalancenames[idxwohub])), size=(800,500), title="Prices", ylabel="€/MWh"))

# # Plot supplies and demands
supplychart = areaplot(x1, supplyvalues,labels=reshape(supplynames,1,length(supplynames)),title="Supply", ylabel = "GWh/h")
demandchart = areaplot(x1, demandvalues,labels=reshape(demandnames,1,length(demandnames)),title="Demand", ylabel = "GWh/h")
# supplychart = areaplot(x1, sum(supplyvalues,dims=2),title="Supply", ylabel = "GWh/h")
# demandchart = areaplot(x1, sum(demandvalues,dims=2),title="Demand", ylabel = "GWh/h")
display(plot([supplychart,demandchart]...,layout=(1,2),size=(800,500)))

# Plot storages
# display(areaplot(x2, hydrolevels1,labels=reshape(hydronames,1,length(hydronames)),size=(800,500),title="Reservoir levels", ylabel = "TWh")) #
display(areaplot(x2, dropdims(sum(hydrolevels1,dims=2),dims=2),labels="Total reservoirs",size=(800,500),title="Reservoir levels", ylabel = "TWh")) #

display(areaplot(x1, dropdims(sum(batterylevels,dims=2),dims=2),labels="Total batteries",size=(800,500),title="Battery levels", ylabel = "GWh")) #

# Plot list of yearly mean production and demand for each supply/demand
meandemand = dropdims(mean(demandvalues,dims=1),dims=1)
meanproduction = dropdims(mean(supplyvalues,dims=1),dims=1)
supplydf = sort(DataFrame(Supplyname = supplynames, Yearly_supply_TWh = meanproduction*8.76),[:Yearly_supply_TWh], rev = true)
demanddf = sort(DataFrame(Demandname = demandnames, Yearly_demand_TWh = meandemand*8.76),[:Yearly_demand_TWh], rev = true)
supplydf[!,:ID] = collect(1:length(supplynames))
demanddf[!,:ID] = collect(1:length(demandnames))
joineddf = select!(outerjoin(supplydf,demanddf;on=:ID),Not(:ID))
show(joineddf,allcols=true, allrows=true, nosubheader = true)

# Check that total supply equals total demand
show(combine(joineddf, [:Yearly_supply_TWh, :Yearly_demand_TWh] .=> sum∘skipmissing), nosubheader = true)

# # Plot list of yearly income and cost for each supply/demand (only works if exogenprices are collected)
# supplyrev = copy(supplyvalues)
# for (i,supplybalancename) in enumerate(supplybalancenames)
#     idx = findfirst(isequal(supplybalancename), powerbalancenames)
#     supplyrev[:,i] .= supplyrev[:,i] .* prices[:,idx]
# end
# demandrev = copy(demandvalues)
# for (i,demandbalancename) in enumerate(demandbalancenames)
#     idx = findfirst(isequal(demandbalancename), powerbalancenames)
#     demandrev[:,i] .= demandrev[:,i] .* prices[:,idx]
# end
# meandemandrev = dropdims(mean(demandrev,dims=1),dims=1)
# meanproductionrev = dropdims(mean(supplyrev,dims=1),dims=1)
# supplyrevdf = sort(DataFrame(Supplyname = supplynames, Yearly_rev_mill€ = meanproductionrev*8.76),[:Yearly_rev_mill€], rev = true)
# demandrevdf = sort(DataFrame(Demandname = demandnames, Yearly_cost_mill€ = meandemandrev*8.76),[:Yearly_cost_mill€], rev = true)
# supplyrevdf[!,:ID] = collect(1:length(supplynames))
# demandrevdf[!,:ID] = collect(1:length(demandnames))
# joinedrevdf = select!(outerjoin(supplyrevdf,demandrevdf;on=:ID),Not(:ID))
# # show(joinedrevdf,allcols=true, allrows=true, nosubheader = true)

# # Sum revenues and cost
# show(combine(joinedrevdf, [:Yearly_rev_mill€, :Yearly_cost_mill€] .=> sum∘skipmissing), nosubheader = true)

### Test inflow mellom detailed og agg

In [ ]:
detailedelements = vcat(exogen,detdseries,detdstructure,thermal,windsol,transm,cons,inflow,fuel,nuclear)
elements = vcat(exogen,aggdetd,thermal,windsol,transm,cons,agginflow,fuel,nuclear)

extraelements = DataElement[]

scenarioyearstart = 1991
scenarioyearstop = 2020
push!(extraelements, getelement(TIMEPERIOD_CONCEPT, "ScenarioTimePeriod", "ScenarioTimePeriod", 
        ("Start", getisoyearstart(scenarioyearstart)), ("Stop", getisoyearstart(scenarioyearstop))))

# cnph = 26
# cnpp = 26
# cpdh = Hour(168*2)
# cpdp = Hour(168*2)
cnph = 24
cnpp = 24
cpdh = Hour(2)
cpdp = Hour(2)
hydro_horizon = SequentialHorizon(cnph, cpdh)
power_horizon = SequentialHorizon(cnpp, cpdp)
set_horizon!(extraelements, "Power", power_horizon)
set_horizon!(extraelements, "Battery", power_horizon)
set_horizon!(extraelements, "Hydro", hydro_horizon);

detailedmodelobjects = values(getmodelobjects(vcat(detailedelements, extraelements)))
modelobjects = values(getmodelobjects(vcat(elements, extraelements)));

In [ ]:
datayearstart = 2024
scenarioyear = 1993
weekstart = 15
progdatatime = getisoyearstart(datayearstart) + Week(weekstart-1)
datatime = getisoyearstart(datayearstart) + Week(weekstart-1)
tnormal = PrognosisTime(progdatatime, datatime, getisoyearstart(scenarioyear) + Week(weekstart-1))
days = 365
parts = days;
hor = SequentialHorizon(days, Day(1))

In [ ]:
function checkagginflow(objects, tnormal, hor; prognosis=true)
    sumenergyinflow = 0
    partsumenergyinflow = zeros(getnumperiods(hor))
    for obj in objects
        if obj isa Balance
            if getinstancename(getid(getcommodity(obj))) == "Hydro"
                enekvglobal = 1.0 # if no energy equivalent, assume inflow is already demoninated in GWh
                if haskey(obj.metadata, GLOBALENEQKEY)
                    enekvglobal = obj.metadata[GLOBALENEQKEY]
                end
                for rhsterm in getrhsterms(obj)
                    if rhsterm.param.param isa PrognosisSeriesParam
                        param = rhsterm.param
                        if prognosis == false
                            param = M3SToMM3Param(MeanSeriesParam(rhsterm.param.param.level, rhsterm.param.param.profile))
                        elseif prognosis == "only prognosis"
                            param = M3SToMM3Param(TwoProductParam(MeanSeriesParam(rhsterm.param.param.level, ConstantTimeVector(1.0)), MeanSeriesParam(rhsterm.param.param.prognosis, ConstantTimeVector(1.0))))
                        end
                        sumenergyinflow += getparamvalue(param, tnormal, MsTimeDelta(getduration(hor)))*enekvglobal
                        for j in 1:getnumperiods(hor)
                            starttime = getstarttime(hor, j, tnormal)
                            timedelta = gettimedelta(hor, j)
                            partsumenergyinflow[j] += getparamvalue(param, starttime, timedelta)*enekvglobal
                        end
                    end
                end
            end
        end
    end
    return sumenergyinflow, partsumenergyinflow
end

In [ ]:
tot1, ser1 = checkagginflow(modelobjects, tnormal, hor, prognosis=false)

In [ ]:
tot2, ser2 = checkagginflow(detailedmodelobjects, tnormal, hor, prognosis=false)

In [ ]:
tot3, ser3 = checkagginflow(modelobjects, tnormal, hor, prognosis="only prognosis")

In [ ]:
tot4, ser4 = checkagginflow(detailedmodelobjects, tnormal, hor, prognosis="only prognosis")

In [ ]:
tot5, ser5 = checkagginflow(modelobjects, tnormal, hor, prognosis=true)

In [ ]:
tot6, ser6 = checkagginflow(detailedmodelobjects, tnormal, hor, prognosis=true)

In [ ]:
display(plot(hcat(ser1, ser2, ser3, ser4, ser5, ser6), lc=[:red :red :blue :blue :green :green]))
println([sum(ser1), sum(ser2)])
println([sum(ser3), sum(ser4)])
println([sum(ser5), sum(ser6)])

In [ ]:
h = Highs_create()
Highs_readModel(h, "failed_model_1.mps")
Highs_setStringOptionValue(h, "solver", "ipm")
Highs_setIntOptionValue(h, "simplex_scale_strategy", 5)
ret = Highs_run(h)